# Federal Tech Employees: Age and Demographic Analysis
## Processing Fed Scope Data
#### US Digtal Corps Analysis - Bench Work 2024

Contributors:
- Colton Lapp (USDC '24 - colton.lapp@gsa.gov)

---

## Overview:
Goal is to understand the demographic breakdown of technologists in the Federal Government using the publicly available "FedScope Employment Data". 

## Data:
The data files are quite large as they contain anonymized individual level information for all individuals in the Federal Workforce (excludes contractors). The data is quarterly until 2010, and then annual from 2010 to 1998. 

By downloading, combining and filtering this data, we are able to create time series trends of the Federal Workforce. 

The data can be downloaded from this [open data website by the OPM](https://www.opm.gov/data/datasets/). Download every file titled "FedScope Employment Cube" and move the zip folder to `data/raw_data`, unzip the file, and rename it as **FedScope_{Month}Year** - i.e. **FedScope_Dec2023**. 

---

### Example Data File Tree

Fed_IT_Employment/

|-- data/

    |-- raw_data.txt

    |-- FedScope_Month_Year

        |-- FACTDATA_MONTHYEAR.TXT

        |-- txt file

        |-- txt file

## Set Up

#### Import Modules

In [108]:
import pandas as pd
import os 
from tqdm import tqdm

#### Set up Pathing

In [109]:
# get root dir which ends in repo_name
repo_name = 'Fed_IT_Employment'
root = os.getcwd()
while os.path.basename(root) != repo_name:
    root = os.path.dirname(root)

# Get raw data directory
rdir = os.path.join( root, 'data', 'raw_data')

print(f"Base directory: {root}\nRaw data directory: {rdir}")

Base directory: /Users/coltonlapp/Dropbox/My Mac (Coltons-MacBook-Pro.local)/Desktop/Work/USDC/publicwork/Fed_IT_Employment
Raw data directory: /Users/coltonlapp/Dropbox/My Mac (Coltons-MacBook-Pro.local)/Desktop/Work/USDC/publicwork/Fed_IT_Employment/data/raw_data


## Set up dictionaries for file processing

In [110]:
# List of years for which data is available, for quarterly data and for September data (Mar, June, Dec are quarterly)

years_sep = [str(y) for y in range(1998, 2024)]
years_quart = [str(y) for y in range(2010, 2024)]


quarter_file_info = [
         { 'month':'MAR', 'years':years_quart},
         { 'month':'JUN', 'years':years_quart},
         { 'month':'SEP', 'years':years_sep},
         { 'month':'DEC', 'years':years_quart}
         ]

filenames = {
        'fedscope_folder' : 'FedScope_{}{}',
        'emp_data' : 'FACTDATA_{month}{year}.TXT',
        'age_codebook' : 'DTagelvl.txt',
        'loc_codebook' : 'DTloc.txt',
        'agency_codebook' : 'DTagy.txt',
        'ed_level_codebook' : 'DTedlvl.txt',
        }   


# Create mapping of column names to a more readble format
colname_map = {
    'AGYSUB' : 'agency',
    'LOC' : 'location',
    'AGELVL' : 'age',
    'EDLVL' : 'ed_level',
    'GSEGRD' : 'gen_sched_eq_grade',
    'LOSLVL' : 'length_of_service',
    'OCC' : 'occupation',
    'PATCO' : 'occupation_category',
    'PPGRD' : 'pay_plan_grade',
    'SALLVL' : 'salary_level',
    'STEMOCC' : 'stem_occ',
    'SUPERVIS' : 'supervisory_status',
    'TOA' : 'type_of_appointment',
    'WORKSCH' : 'work_schedule',
    'WORKSTAT' : 'work_status',
    'DATECODE' : 'date',
    'EMPLOYMENT' : 'employment',
    'SALARY' : 'avg_salary',
    'LOS' : 'avg_length_of_service'
}



### Functions to map categorically coded variables to readable values

Many categorical variables in FedScope employment data is categorical and represented as numbers/letters to make the files smaller. The codebook is then saved in a seperate file. These helper functions read in those codebook files and then convert the categorical columns to contain the human readable labels. 

In [111]:
def map_age(df, folder):
    """
    This function maps the AGELVL column to a more readable format using the age_codebook file
    """
    age_codebook = pd.read_csv(os.path.join( rdir, folder, filenames['age_codebook']))
    
    # turn age_codebook df into a dictionary mapping  column AGELVL to column AGELVLT
    age_dict = dict(zip(age_codebook['AGELVL'], age_codebook['AGELVLT']))

    # map AGELVL to AGELVLT
    df['age'] = df['age'].map(age_dict)
    return df

def map_location(df, folder):
    """
    This function maps the LOC column to a more readable format using the loc_codebook file
    """
    loc_codebook = pd.read_csv(os.path.join( rdir, folder, filenames['loc_codebook']))
    
    # turn loc_codebook df into a dictionary mapping  column LOC to column LOC_NAME
    loc_dict = dict(zip(loc_codebook['LOC'], loc_codebook['LOCT']))

    # map LOC to LOCT
    df.loc[:, 'location'] = df['location'].map(loc_dict)

    return df

def map_agency(df, folder):
    """
    This function uses AGYSUB to create two new columns by merging, AGYT and AGYSUBT
    """
    agency_codebook =  pd.read_csv(os.path.join( rdir, folder, filenames['agency_codebook']))
    agency_codebook = agency_codebook[['AGYSUB', 'AGYT', 'AGYSUBT']]

    # rename AGYSUB to agency
    agency_codebook.rename(columns={'AGYSUB':'agency'}, inplace=True)

    # drop duplicates
    agency_codebook.drop_duplicates(inplace=True)

    # Join columns AGYT and AGYSUBT
    df = pd.merge(df, agency_codebook, on='agency', how='left')

    # drop AGYSUB
    df.drop(columns='agency', inplace=True)


    # Rename AGYT to agency, AGYSUBT to agency_sub
    df.rename(columns={'AGYT':'agency', 'AGYSUBT':'agency_sub'}, inplace=True)

    return df

def map_ed_level(df, folder):
    """
    This function uses the EDLVL column to merge to two more columns: EDLVLTYPT and EDLVLT
    """
    ed_level_codebook = pd.read_csv(os.path.join( rdir, folder, filenames['ed_level_codebook']))
    ed_level_codebook = ed_level_codebook[['EDLVL', 'EDLVLTYPT', 'EDLVLT']]

    # rename EDLVL to ed_level
    ed_level_codebook.rename(columns={'EDLVL':'ed_level'}, inplace=True)

    # drop duplicates
    ed_level_codebook.drop_duplicates(inplace=True)

    # Join columns EDLVLTYPT and EDLVLT
    df = pd.merge(df, ed_level_codebook, on='ed_level', how='left')

    # drop EDLVL
    df.drop(columns='ed_level', inplace=True)

    # Rename EDLVLTYPT to ed_level, EDLVLT to ed_level_detailed
    df.rename(columns={'EDLVLTYPT':'ed_level', 'EDLVLT':'ed_level_detailed'}, inplace=True)

    return df

def map_occupation(df, folder):
    """
    This function maps the OCC column to a more readable format using the occ_codebook file
    """
    occ_codebook = pd.read_csv(os.path.join( rdir, folder, filenames['occ_codebook']))
    occ_codebook = occ_codebook[['OCC','OCCT','OCCTYPT']]
    
    # rename OCC to occupation
    occ_codebook.rename(columns={'OCC':'occupation'}, inplace=True)

    # drop duplicates
    occ_codebook.drop_duplicates(inplace=True)

    # Join columns OCCT and OCCTYPT
    df = pd.merge(df, occ_codebook, on='occupation', how='left')

    # drop OCC
    df.drop(columns='occupation', inplace=True)

    # Rename OCCT to occupation, OCCTYPT to occupation_type
    df.rename(columns={'OCCT':'occupation', 'OCCTYPT':'occupation_type'}, inplace=True)

    return df


def map_date(df):
    # convert date to datetime with format YYYYMM
    df.loc[:, 'date'] = pd.to_datetime(df['date'], format='%Y%m')

    # return
    return df

## Read in all datasets, convert categorical columns, append together

In [112]:
# Columns we want to keep in processed dataset
cols_keep = ['agency', 'location', 'age', 'ed_level', 'occupation', 'occupation_category', 'pay_plan_grade', 'date']

# Container for concatenated data
concat_df = pd.DataFrame()

# Loop through each quarter and year, read in the data, and concatenate to concat_df
for my_dict in quarter_file_info:
  for year in tqdm(my_dict['years'], desc=f"Processing years for quarter ending in month {my_dict['month']}"):

      # get folder and filename
      folder = filenames['fedscope_folder'].format(my_dict['month'].title(), year)
      fname = filenames['emp_data'].format(month=my_dict['month'], year=year)

      # read in data
      df = pd.read_csv(os.path.join(rdir, folder, fname), low_memory=False)

      # rename df columns
      df.rename(columns=colname_map, inplace=True)
      
      # subset columns of interest
      df = df[cols_keep]

      # Convert age, date, and location columns to more readable format
      df = map_age(df, folder)
      df = map_date(df)
      df = map_location(df, folder)
      df = map_agency(df, folder)
      df = map_ed_level(df, folder)
      

      # concat df to concat_df
      concat_df = pd.concat([concat_df, df], axis=0)




Processing years for quarter ending in month DEC: 100%|██████████| 14/14 [02:53<00:00, 12.42s/it]


### Concatenated, Processed, and Mapped Final Dataset to Save

In [113]:
# Show the first few rows of the concatenated data
concat_df 

,location,age,occupation,occupation_category,pay_plan_grade,date,agency,agency_sub,ed_level,ed_level_detailed
0,11-DISTRICT OF COLUMBIA,65 or more,0301,2,EX-02,2010-03-01,AA-ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...,DOCTORATE,21-DOCTORATE DEGREE
1,IT-ITALY,60-64,1630,2,GS-12,2010-03-01,AB-AMERICAN BATTLE MONUMENTS COMMISSION,AB00-AMERICAN BATTLE MONUMENTS COMMISSION,BETWEEN HS & BACHELORS,07-SOME COLLEGE - LESS THAN ONE YEAR
2,FR-FRANCE,50-54,1630,2,GS-12,2010-03-01,AB-AMERICAN BATTLE MONUMENTS COMMISSION,AB00-AMERICAN BATTLE MONUMENTS COMMISSION,HIGH SCHOOL OR EQUIVALENCY,04-HIGH SCHOOL GRADUATE OR CERTIFICATE OF EQUI...
3,BE-BELGIUM,60-64,1630,2,GS-12,2010-03-01,AB-AMERICAN BATTLE MONUMENTS COMMISSION,AB00-AMERICAN BATTLE MONUMENTS COMMISSION,HIGH SCHOOL OR EQUIVALENCY,04-HIGH SCHOOL GRADUATE OR CERTIFICATE OF EQUI...
4,FR-FRANCE,55-59,0301,2,GS-15,2010-03-01,AB-AMERICAN BATTLE MONUMENTS COMMISSION,AB00-AMERICAN BATTLE MONUMENTS COMMISSION,MASTERS,17-MASTER'S DEGREE
...,...,...,...,...,...,...,...,...,...,...
2261277,11-DISTRICT OF COLUMBIA,30-34,0301,2,AD-00,2023-12-01,ZS-UNITED STATES-CHINA ECONOMIC AND SECURITY R...,ZS00-UNITED STATES-CHINA ECONOMIC AND SECURITY...,HIGH SCHOOL OR EQUIVALENCY,04-HIGH SCHOOL GRADUATE OR CERTIFICATE OF EQUI...
2261278,11-DISTRICT OF COLUMBIA,25-29,0301,2,AD-00,2023-12-01,ZS-UNITED STATES-CHINA ECONOMIC AND SECURITY R...,ZS00-UNITED STATES-CHINA ECONOMIC AND SECURITY...,BACHELORS,13-BACHELOR'S DEGREE
2261279,11-DISTRICT OF COLUMBIA,30-34,0301,2,AD-00,2023-12-01,ZS-UNITED STATES-CHINA ECONOMIC AND SECURITY R...,ZS00-UNITED STATES-CHINA ECONOMIC AND SECURITY...,HIGH SCHOOL OR EQUIVALENCY,04-HIGH SCHOOL GRADUATE OR CERTIFICATE OF EQUI...
2261280,51-VIRGINIA,35-39,0301,2,AD-00,2023-12-01,ZS-UNITED STATES-CHINA ECONOMIC AND SECURITY R...,ZS00-UNITED STATES-CHINA ECONOMIC AND SECURITY...,MASTERS,17-MASTER'S DEGREE


#### Write merged and processed dataset to file

In [114]:
concat_df.to_csv(os.path.join(root, 'data', 'processed_data', 'FedScope_emp_data_concat.csv'), index=False)